In [ ]:
!pip install unsloth bitsandbytes transformers

In [ ]:
from unsloth import FastLanguageModel

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)


In [ ]:
sample_data = {
    "Live_data_from_frontend": """User Info:
user_name = Anika, email = anika@gmail.com, orders = 12, spent = 1543, email_verified = anika@gmail.com, refunds = 595, days_since_last = 228

Product Info:
product_name = Toe Ring, product_description = Simple metal toe rings in sterling silver or gold-plated finishes., product_color = Carmine, product_size = One Size, product_price = 706 INR, min_price_client = 367 INR, avg_inventory = 27, product_tag = spring, product_type = regular, images_count = 6, days_since_creation = 183""",
    "user": "User: 10% is less it seems is that all you can offer,ok lets say if you can make it to 25 thats a deal",
    "response": "Bot: Id love to work something out how about 15% off, making it 600.1 INR? I'm open to hearing your offer too!",
    "metadata": {
        "user_email_id": "anika@gmail.com",
        "user_score": 21.55011282634933,
        "bargain_score": 40,
        "product_price": 706,
        "product_score": 18.7020694468137,
        "admin_max_discount_from_min_price": 48,
        "system_max_discount": 24.0,
        "max_round_count": 5,
        "current_round": 2,
        "previous_round_discount_given": 10,
        "user_req": 25,
        "current_discount": 15,
        "current_discount_price": 600.1,
        "round_discounts": {
            "round_count1": 10,
            "round_count2": 15,
            "round_count3": 19,
            "round_count4": 21,
            "round_count5": 24
        }
    }
}

In [ ]:
import re

def parse_live_data(data_str):
    try:
        _, rest = data_str.split("User Info:", 1)
        user_part, product_part = rest.split("Product Info:", 1)

        def parse_section(section):
            pattern = r"(\w+[\s\S]*?)\s*=\s*(.+?)(?=(?:,\s*\w+[\s\S]*?\s*=)|$)"
            matches = re.findall(pattern, section.strip(), re.DOTALL)
            return {k.strip(): v.strip() for k, v in matches}

        return {"user": parse_section(user_part), "product": parse_section(product_part)}
    except Exception as e:
        return {"user": {}, "product": {}}

# Run parser
parsed = parse_live_data(sample_data["Live_data_from_frontend"])

# Print results
print("User Data:")
for k, v in parsed["user"].items():
    print(f"{k} = {v}")

print("\nProduct Data:")
for k, v in parsed["product"].items():
    print(f"{k} = {v}")

In [ ]:
def flatten_dict(d, parent_key='', sep='.'):
    """Flatten a nested dictionary for metadata."""
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

# Flatten the metadata
flattened_metadata = flatten_dict(sample_data["metadata"])

# Print flattened metadata
print("=== FLATTENED METADATA ===")
for key, value in flattened_metadata.items():
    print(f"{key} = {value}")

# Print conversation
print("\n=== CONVERSATION ===")
print(sample_data["user"])
print(sample_data["response"])

In [ ]:
def safe_cast(val, to_type, default):
    """Safely cast a value to a type, returning default if it fails."""
    try:
        return to_type(val)
    except (ValueError, TypeError):
        return default



def build_context(entry):
    """Build a flat context dict from a single entry in final_output.json, using all parameters."""
    live_data_str = entry.get("Live_data_from_frontend", "")
    parsed = parse_live_data(live_data_str)
    user_info = parsed.get("user", {})
    product_info = parsed.get("product", {})
    metadata = entry.get("metadata", {})  # Don't flatten metadata yet

    # Extract round_discounts before flattening
    round_discounts = metadata.get("round_discounts", {})

    # Now flatten the rest of metadata
    flattened_metadata = flatten_dict(metadata)

    # Build context with all possible parameters
    context = {}

    # Add all user/product info
    for k, v in user_info.items():
        context[f"user_{k}"] = v
    for k, v in product_info.items():
        context[f"product_{k}"] = v

    # Add all metadata (flattened)
    for k, v in flattened_metadata.items():
        if k != 'round_discounts':  # Skip round_discounts as it's handled separately
            context[f"meta_{k}"] = v

    # Add round_discounts directly to context
    context["round_discounts"] = round_discounts

    # Rest of your build_context function remains the same...
    context["user_input"] = entry.get("user", "").replace("User: ", "").strip()
    context["gold_response"] = entry.get("response", "").replace("Bot: ", "").strip()

    # Some fields for prompt logic (with safe casting)
    context["orders"] = safe_cast(user_info.get("orders", 0), int, 0)
    context["spent"] = safe_cast(user_info.get("spent", 0), int, 0)
    context["refunds"] = safe_cast(user_info.get("refunds", 0), int, 0)
    context["days_since_last"] = safe_cast(user_info.get("days_since_last", 0), int, 0)
    context["email_verified"] = str(user_info.get("email_verified", "False")).lower() in ("true", "1", "yes")

    context["product_name"] = product_info.get("product_name", "Unknown Product")
    context["product_description"] = product_info.get("product_description", "")
    context["product_price"] = safe_cast(str(product_info.get("product_price", "0")).replace(" INR", ""), int, 0)
    context["min_price_client"] = safe_cast(str(product_info.get("min_price_client", "0")).replace(" INR", ""), int, 0)
    context["avg_inventory"] = safe_cast(product_info.get("avg_inventory", 0), int, 0)
    context["product_tag"] = product_info.get("product_tag", "")
    context["product_type"] = product_info.get("product_type", "")
    context["images_count"] = safe_cast(product_info.get("images_count", 0), int, 0)
    context["days_since_creation"] = safe_cast(product_info.get("days_since_creation", 0), int, 0)

    # Metadata fields for prompt logic (with safe casting)
    context["user_score"] = safe_cast(metadata.get("user_score", 0), float, 0)
    context["bargain_score"] = safe_cast(metadata.get("bargain_score", 0), float, 0)
    context["product_score"] = safe_cast(metadata.get("product_score", 0), float, 0)
    context["admin_max_discount_from_min_price"] = safe_cast(metadata.get("admin_max_discount_from_min_price", 0), float, 0)
    context["system_max_discount"] = safe_cast(metadata.get("system_max_discount", 0), float, 0)
    context["max_round_count"] = safe_cast(metadata.get("max_round_count", 0), int, 0)
    context["current_round"] = safe_cast(metadata.get("current_round", 0), int, 0)
    context["previous_round_discount_given"] = safe_cast(metadata.get("previous_round_discount_given", 0), float, 0)
    context["user_req"] = safe_cast(metadata.get("user_req", 0), float, 0)
    context["current_discount"] = safe_cast(metadata.get("current_discount", 0), float, 0)
    context["current_discount_price"] = safe_cast(metadata.get("current_discount_price", 0), float, 0)

    return context

# Build and print the context
context = build_context(sample_data)
print("=== CONTEXT DICTIONARY ===")
for key, value in context.items():
    if key == 'round_discounts':
        print("\nRound Discounts:")
        for rd_key, rd_value in sorted(value.items()):
            print(f"  {rd_key}: {rd_value}%")
    else:
        print(f"{key}: {value}")

In [ ]:
import json
import re
from transformers import TextGenerationPipeline

pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, framework="pt")

def parse_live_data(data_str):
    """Parse the Live_data_from_frontend string into user and product dicts."""
    try:
        _, rest = data_str.split("User Info:", 1)
        user_part, product_part = rest.split("Product Info:", 1)
        def parse_section(section):
            pattern = r"(\w+[\s\S]*?)\s*=\s*(.+?)(?=(?:,\s*\w+[\s\S]*?\s*=)|$)"
            matches = re.findall(pattern, section.strip(), re.DOTALL)
            return {k.strip(): v.strip() for k, v in matches}
        return {"user": parse_section(user_part), "product": parse_section(product_part)}
    except Exception as e:
        print(f"[ERROR] Failed to parse Live_data_from_frontend: {e}")
        return {"user": {}, "product": {}}

def flatten_dict(d, parent_key='', sep='.'):
    """Flatten a nested dictionary for metadata."""
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

def safe_cast(val, to_type, default):
    try:
        return to_type(val)
    except Exception:
        return default

def build_context(entry):
    """Build a flat context dict from a single entry in final_output.json, using all parameters."""
    live_data_str = entry.get("Live_data_from_frontend", "")
    parsed = parse_live_data(live_data_str)
    user_info = parsed.get("user", {})
    product_info = parsed.get("product", {})
    metadata = flatten_dict(entry.get("metadata", {}))

    # Build context with all possible parameters
    context = {}

    # Add all user/product info
    for k, v in user_info.items():
        context[f"user_{k}"] = v
    for k, v in product_info.items():
        context[f"product_{k}"] = v

    # Add all metadata (flattened)
    for k, v in metadata.items():
        context[f"meta_{k}"] = v

    # Add top-level fields
    context["user_input"] = entry.get("user", "").replace("User: ", "").strip()
    context["gold_response"] = entry.get("response", "").replace("Bot: ", "").strip()

    # Some fields for prompt logic (with safe casting)
    context["orders"] = safe_cast(user_info.get("orders", 0), int, 0)
    context["spent"] = safe_cast(user_info.get("spent", 0), int, 0)
    context["refunds"] = safe_cast(user_info.get("refunds", 0), int, 0)
    context["days_since_last"] = safe_cast(user_info.get("days_since_last", 0), int, 0)
    context["email_verified"] = str(user_info.get("email_verified", "False")).lower() in ("true", "1", "yes")

    context["product_name"] = product_info.get("product_name", "Unknown Product")
    context["product_description"] = product_info.get("product_description", "")
    context["product_price"] = safe_cast(str(product_info.get("product_price", "0")).replace(" INR", ""), int, 0)
    context["min_price_client"] = safe_cast(str(product_info.get("min_price_client", "0")).replace(" INR", ""), int, 0)
    context["avg_inventory"] = safe_cast(product_info.get("avg_inventory", 0), int, 0)
    context["product_tag"] = product_info.get("product_tag", "")
    context["product_type"] = product_info.get("product_type", "")
    context["images_count"] = safe_cast(product_info.get("images_count", 0), int, 0)
    context["days_since_creation"] = safe_cast(product_info.get("days_since_creation", 0), int, 0)

    # Metadata fields for prompt logic (with safe casting)
    context["user_score"] = safe_cast(metadata.get("user_score", 0), float, 0)
    context["bargain_score"] = safe_cast(metadata.get("bargain_score", 0), float, 0)
    context["product_score"] = safe_cast(metadata.get("product_score", 0), float, 0)
    context["admin_max_discount_from_min_price"] = safe_cast(metadata.get("admin_max_discount_from_min_price", 0), float, 0)
    context["system_max_discount"] = safe_cast(metadata.get("system_max_discount", 0), float, 0)
    context["max_round_count"] = safe_cast(metadata.get("max_round_count", 0), int, 0)
    context["current_round"] = safe_cast(metadata.get("current_round", 0), int, 0)
    context["previous_round_discount_given"] = safe_cast(metadata.get("previous_round_discount_given", 0), float, 0)
    context["user_req"] = safe_cast(metadata.get("user_req", 0), float, 0)
    context["current_discount"] = safe_cast(metadata.get("current_discount", 0), float, 0)
    context["current_discount_price"] = safe_cast(metadata.get("current_discount_price", 0), float, 0)
    context["round_discounts"] = metadata.get("round_discounts", {})

    return context

def get_messages(context):
    # Parameter influence summary (as in your prompt)
    parameter_summary = f"""
Parameter: Orders → More orders → Higher discount → {context['orders']}
Parameter: Total Spent → More money spent → Higher discount → ₹{context['spent']}
Parameter: Days Since Last Purchase → Longer time since last purchase → Higher discount → {context['days_since_last']} days
Parameter: Inventory Level → More inventory → Higher discount → {context['avg_inventory']} units
Parameter: Days Since Creation → Older product → Higher discount → {context['days_since_creation']} days
Parameter: Image Count → More images → Higher discount → {context['images_count']}
Parameter: Bargain Score → Higher score → Higher discount → {context['bargain_score']}
Parameter: Max Round Count → More rounds available → Can afford higher discount later → {context['max_round_count']}

Parameter: Refunds → More refunds → Lower discount → ₹{context['refunds']}
Parameter: Current Round → Later rounds can go less high → Higher round number → Lower discount allowed → Round {context['current_round']}
Parameter: Product Price → Higher price → Lower discount allowed → ₹{context['product_price']}
Parameter: Min Price Client → Higher minimum → Less room for discount → ₹{context['min_price_client']}
Parameter: System Max Discount → Lower max → Lower discount allowed → {context['system_max_discount']}%
"""

    user_prompt = f"""
{parameter_summary}

User Profile:
- Orders: {context['orders']}
- Total Spent: ₹{context['spent']}
- Refunds: ₹{context['refunds']}
- Days Since Last Purchase: {context['days_since_last']}
- Email Verified: {"Yes" if context['email_verified'] else "No"}

Product Details:
- Inventory Level: {context['avg_inventory']} units
- Age: {context['days_since_creation']} days old
- Image Count: {context['images_count']}
- Tag: {context['product_tag']}
- Type: {context['product_type']}

Negotiation Info:
- You're on Round {context['current_round']} of {context['max_round_count']}
- Maximum Allowed Discount: {context['system_max_discount']}%
- Last Offer: {context['previous_round_discount_given']}%
- Customer wants more than the last offer

The customer said: "{context['user_input']}"

Based on this, suggest a new discount for this round only.
Don't jump straight to the maximum — save room for future rounds.

Respond with only one persuasive line that includes:
- The discount percentage
- The final price after discount
"""

    system_prompt = f"""
You are an intelligent shopping assistant negotiating prices over multiple rounds.
Your goal is to reach a deal while strategically managing discounts across all rounds.

Rules:
1. You're currently in Round {context['current_round']} of {context['max_round_count']}.
2. Your maximum allowed discount is {context['system_max_discount']}%.
3. Don't jump straight to the highest discount — leave room for future rounds.
4. Suggest a reasonable increase from the last offer: {context['previous_round_discount_given']}%.
5. Always include: discount %, final price, optional negotiation invite.
6. Respond with only one persuasive line — no extra text or explanation.
7. And discount should always be less or equal to {context['system_max_discount']}%
"""

    return [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": user_prompt.strip()}
    ]

def extract_discount(text):
    """Extract the discount percentage from the model's response (best effort)."""
    match = re.search(r'(\d+(\.\d+)?)\s*%', text)
    if match:
        return float(match.group(1))
    return None

def generate_responses(input_file_path, output_file_path, limit=50):
    with open(input_file_path, "r", encoding="utf-8") as f:
        input_data = json.load(f)
    results = []
    for idx, entry in enumerate(input_data[:limit]):
        print(f"Processing scenario {idx + 1}/{limit}")
        context = build_context(entry)
        messages = get_messages(context)
        prompt = tokenizer.apply_chat_template(messages, tokenize=False)
        outputs = pipeline(
            prompt,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
        bot_response = outputs[0]["generated_text"][len(prompt):].strip()
        discount_pred = extract_discount(bot_response)
        # Append predicted discount to response
        full_bot_response = f"Bot: {bot_response}"
        if discount_pred is not None:
            full_bot_response += f" [Predicted Discount: {discount_pred}%]"
        # Copy original entry, update response and gold
        updated_entry = entry.copy()
        updated_entry["gold"] = entry.get("response", "")     # Original Bot response
        updated_entry["response"] = full_bot_response         # New model-generated response
        results.append(updated_entry)
    with open(output_file_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4, ensure_ascii=False)
    print(f"✅ Saved {len(results)} responses to {output_file_path}")

# Example usage (uncomment to run):
generate_responses("final_output.json", "responses.json", limit=10)

In [ ]:
from transformers import TextGenerationPipeline
import json

def generate_discount_prompt(context):
    # Calculate derived values
    min_price_pct = (context['min_price'] / context['product_price']) * 100
    refund_rate = (context['refunds'] / context['total_spent']) * 100 if context['total_spent'] > 0 else 0
    current_price = context['product_price'] * (1 - context['current_discount']/100)
    next_round = min(context['current_round'] + 1, context['max_rounds'])

    return f"""**DISCOUNT CALCULATION FRAMEWORK**

**1. SCENARIO ANALYSIS**

**User Profile:**
- **Orders**: {context['orders']} ({'New' if context['orders'] < 5 else 'Medium' if context['orders'] < 20 else 'Loyal'})
- **Total Spent**: ₹{context['total_spent']:,} ({'Low' if context['total_spent'] < 1000 else 'Medium' if context['total_spent'] < 5000 else 'High'})
- **Refund Rate**: {refund_rate:.1f}% ({'Low' if refund_rate < 10 else 'Medium' if refund_rate < 25 else 'High'} risk)
- **Last Active**: {context['days_since_last']} days ({'Recent' if context['days_since_last'] < 30 else 'Average' if context['days_since_last'] < 90 else 'Inactive'})
- **Bargain Skill**: {context['bargain_score']}/100 ({'Weak' if context['bargain_score'] < 40 else 'Average' if context['bargain_score'] < 70 else 'Strong'})

**Product Status:**
- **Price**: ₹{context['product_price']}
- **Min Price**: ₹{context['min_price']} ({min_price_pct:.0f}% of original)
- **Inventory**: {context['inventory']} units ({'Critical' if context['inventory'] < 5 else 'Low' if context['inventory'] < 20 else 'Normal'})
- **Age**: {context['product_age']} days ({'New' if context['product_age'] < 30 else 'Average' if context['product_age'] < 180 else 'Old'})
- **Type**: {context['product_type']}
- **Tag**: {context['product_tag']}

**Negotiation State:**
- **Round**: {context['current_round']}/{context['max_rounds']}
- **Previous Discount**: {context['prev_discount']}%
- **Current Offer**: {context['current_discount']}% (₹{current_price:.2f})
- **User's Ask**: {context['user_ask']}%
- **System Max**: {context['system_max_discount']}%

**2. DECISION FACTORS**

**A. User Value (Score: {context['user_score']:.1f}/45)**
- Tier: {'Low' if context['user_score'] < 16 else 'Medium' if context['user_score'] < 31 else 'High'}
- Adjustment: {0.5 if context['user_score'] < 16 else 1 if context['user_score'] < 31 else 1.5}-{1 if context['user_score'] < 16 else 1.5 if context['user_score'] < 31 else 2}%

**B. Inventory Status**
- Level: {'Critical' if context['inventory'] < 5 else 'Low' if context['inventory'] < 20 else 'Normal'}
- Adjustment: {2 if context['inventory'] < 5 else 1 if context['inventory'] < 20 else 0.5}-{3 if context['inventory'] < 5 else 2 if context['inventory'] < 20 else 1}%

**C. Product Age**
- Category: {'New' if context['product_age'] < 30 else 'Average' if context['product_age'] < 180 else 'Old'}
- Adjustment: {0 if context['product_age'] < 30 else 0.5 if context['product_age'] < 180 else 1}-{0.5 if context['product_age'] < 30 else 1 if context['product_age'] < 180 else 2}%

**D. Bargain Score**
- Level: {'Weak' if context['bargain_score'] < 40 else 'Average' if context['bargain_score'] < 70 else 'Strong'}
- Adjustment: {0 if context['bargain_score'] < 40 else 0.5 if context['bargain_score'] < 70 else 1}-{1 if context['bargain_score'] < 40 else 1.5 if context['bargain_score'] < 70 else 2}%

**E. Round Progression**
- Phase: {'Early' if context['current_round'] <= 2 else 'Middle' if context['current_round'] <= 4 else 'Final'}
- Expected Increase: {2 if context['current_round'] <= 2 else 1 if context['current_round'] <= 4 else 0.5}-{3 if context['current_round'] <= 2 else 2 if context['current_round'] <= 4 else 1}%

**3. DISCOUNT CALCULATION**

**Current Round ({context['current_round']}/{context['max_rounds']}):**
- Base: {context['prev_discount']}%
- New Offer: {context['current_discount']}%
- Price: ₹{current_price:.2f}

**Future Rounds:**
1. Calculate discount for each remaining round
2. Ensure progression is logical (increasing discounts)
3. Final round should approach system max discount
4. All prices must be ≥ ₹{context['min_price']}

**4. OUTPUT FORMAT**
{{
  "discount_progression": {{
    "round_{context['current_round']}": "{context['current_discount']}% (₹{current_price:.2f})",
    "round_{context['current_round'] + 1}": "X% (₹Y)",
    "round_{context['current_round'] + 2}": "Z% (₹W)",
    ... (for all remaining rounds)
    "round_{context['max_rounds']}": "{context['system_max_discount']}% (₹{context['product_price'] * (1 - context['system_max_discount']/100):.2f})"
  }},
  "reasoning": "Explanation of discount progression logic",
  "constraints": {{
    "min_price": {context['min_price']},
    "system_max": {context['system_max_discount']}%,
    "current_round": {context['current_round']}/{context['max_rounds']}
  }}
}}

**5. EXAMPLE RESPONSE**
{{
  "discount_progression": {{
    "round_3": "11% (₹581.17)",
    "round_4": "12% (₹574.64)",
    "round_5": "13.5% (₹564.85)",
    "round_6": "15% (₹555.05)"
  }},
  "reasoning": "Gradual increase based on inventory status (critical) and user value (high). Final round reaches system max discount.",
  "constraints": {{
    "min_price": 496,
    "system_max": 15%,
    "current_round": "3/6"
  }}
}}"""

def calculate_discount_progression(context):
    """Calculate a logical discount progression from current round to max round"""
    progression = {}

    # Add current round
    current_price = context['product_price'] * (1 - context['current_discount']/100)
    progression[f"round_{context['current_round']}"] = f"{context['current_discount']}% (₹{current_price:.2f})"

    # Calculate remaining rounds
    remaining_rounds = context['max_rounds'] - context['current_round']
    if remaining_rounds <= 0:
        return progression

    # Calculate discount increment per round
    discount_gap = context['system_max_discount'] - context['current_discount']

    # Determine if we should use linear or non-linear progression
    if context['inventory'] < 5 or context['user_score'] > 30:
        # For critical inventory or high-value users, front-load discounts
        increments = []
        for i in range(remaining_rounds):
            if i == remaining_rounds - 1:  # Last round
                increments.append(discount_gap - sum(increments))
            else:
                # Front-loaded progression
                round_increment = discount_gap * (0.4 - 0.05 * i) / (remaining_rounds - 1)
                increments.append(round_increment)
    else:
        # Linear progression for normal cases
        increment_per_round = discount_gap / remaining_rounds
        increments = [increment_per_round] * remaining_rounds

    # Apply increments to calculate future discounts
    current_discount = context['current_discount']
    for i in range(remaining_rounds):
        round_num = context['current_round'] + i + 1
        current_discount += increments[i]
        # Round to nearest 0.5
        rounded_discount = round(current_discount * 2) / 2
        # Ensure we don't exceed system max
        rounded_discount = min(rounded_discount, context['system_max_discount'])

        price = context['product_price'] * (1 - rounded_discount/100)
        progression[f"round_{round_num}"] = f"{rounded_discount}% (₹{price:.2f})"

    return progression

def generate_discount_response(context, model, tokenizer):
    # Generate the prompt using the context
    prompt = generate_discount_prompt(context)

    # Format the messages for the chat template
    messages = [
        {"role": "system", "content": "You are a helpful pricing assistant that determines optimal discount offers."},
        {"role": "user", "content": prompt}
    ]

    # Apply chat template
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

    # Initialize the pipeline
    pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, framework="pt")

    # Generate response
    outputs = pipeline(
        formatted_prompt,
        max_new_tokens=300,  # Increased for detailed responses
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    # Extract just the assistant's response
    bot_response = outputs[0]["generated_text"][len(formatted_prompt):].strip()

    # Extract JSON from response
    json_start = bot_response.find('{')
    json_end = bot_response.rfind('}') + 1

    if json_start >= 0 and json_end > 0:
        try:
            parsed = json.loads(bot_response[json_start:json_end])

            # Ensure we have a complete discount progression
            if 'discount_progression' in parsed:
                # Check if all rounds are included
                has_all_rounds = True
                for r in range(context['current_round'], context['max_rounds'] + 1):
                    if f"round_{r}" not in parsed['discount_progression']:
                        has_all_rounds = False
                        break

                if has_all_rounds:
                    return parsed

            # If we don't have a complete progression, fall back to calculated one
            parsed['discount_progression'] = calculate_discount_progression(context)
            return parsed

        except:
            pass

    # If JSON parsing fails, create a fallback response
    progression = calculate_discount_progression(context)

    # Extract next round discount for convenience
    next_round = context['current_round'] + 1
    next_discount = None
    if f"round_{next_round}" in progression:
        discount_text = progression[f"round_{next_round}"]
        next_discount = float(discount_text.split('%')[0])

    return {
        "discount_progression": progression,
        "reasoning": "Calculated discount progression based on user value, inventory status, and round progression.",
        "constraints": {
            "min_price": context['min_price'],
            "system_max": context['system_max_discount'],
            "current_round": f"{context['current_round']}/{context['max_rounds']}"
        }
    }

# Example usage
if __name__ == "__main__":




    # Example context

    example_context = {
    # User
    'orders': 12,
    'total_spent': 1543,
    'refunds': 595,
    'days_since_last': 228,
    'bargain_score': 40,
    'user_score': 21.55,

    # Product
    'product_price': 706,
    'min_price': 367,
    'inventory': 27,
    'product_age': 183,
    'product_type': 'regular',
    'product_tag': 'spring',

    # Negotiation
    'current_round': 2,
    'max_rounds': 5,
    'prev_discount': 10,
    'current_discount': 18,
    'user_ask': 25,
    'system_max_discount': 24.0
    }

    # Generate discount response
    response = generate_discount_response(example_context, model, tokenizer)

    # Print the discount information
    print(f"Current Round: {example_context['current_round']}/{example_context['max_rounds']}")
    print(f"Current Discount: {example_context['current_discount']}%")
    print(f"Current Price: ₹{example_context['product_price'] * (1 - example_context['current_discount']/100):.2f}")

    # Print discount progression
    print("\nDiscount Progression:")
    for round_key, round_value in response.get('discount_progression', {}).items():
        print(f"{round_key}: {round_value}")

    print(f"\nReasoning: {response.get('reasoning')}")


NEW PROMPT TEMPLATE 1

In [ ]:
import json
import re
from transformers import TextGenerationPipeline

pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, framework="pt")

def parse_live_data(data_str):
    """Parse the Live_data_from_frontend string into user and product dicts."""
    try:
        _, rest = data_str.split("User Info:", 1)
        user_part, product_part = rest.split("Product Info:", 1)
        def parse_section(section):
            pattern = r"(\w+[\s\S]*?)\s*=\s*(.+?)(?=(?:,\s*\w+[\s\S]*?\s*=)|$)"
            matches = re.findall(pattern, section.strip(), re.DOTALL)
            return {k.strip(): v.strip() for k, v in matches}
        return {"user": parse_section(user_part), "product": parse_section(product_part)}
    except Exception as e:
        print(f"[ERROR] Failed to parse Live_data_from_frontend: {e}")
        return {"user": {}, "product": {}}

def flatten_dict(d, parent_key='', sep='.'):
    """Flatten a nested dictionary for metadata."""
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

def safe_cast(val, to_type, default):
    try:
        return to_type(val)
    except Exception:
        return default

def build_context(entry):
    """Build a flat context dict from a single entry in final_output.json, using all parameters."""
    live_data_str = entry.get("Live_data_from_frontend", "")
    parsed = parse_live_data(live_data_str)
    user_info = parsed.get("user", {})
    product_info = parsed.get("product", {})
    metadata = flatten_dict(entry.get("metadata", {}))

    # Build context with all possible parameters
    context = {}

    # Add all user/product info
    for k, v in user_info.items():
        context[f"user_{k}"] = v
    for k, v in product_info.items():
        context[f"product_{k}"] = v

    # Add all metadata (flattened)
    for k, v in metadata.items():
        context[f"meta_{k}"] = v

    # Add top-level fields
    context["user_input"] = entry.get("user", "").replace("User: ", "").strip()
    context["gold_response"] = entry.get("response", "").replace("Bot: ", "").strip()

    # Some fields for prompt logic (with safe casting)
    context["orders"] = safe_cast(user_info.get("orders", 0), int, 0)
    context["spent"] = safe_cast(user_info.get("spent", 0), int, 0)
    context["refunds"] = safe_cast(user_info.get("refunds", 0), int, 0)
    context["days_since_last"] = safe_cast(user_info.get("days_since_last", 0), int, 0)
    context["email_verified"] = str(user_info.get("email_verified", "False")).lower() in ("true", "1", "yes")

    context["product_name"] = product_info.get("product_name", "Unknown Product")
    context["product_description"] = product_info.get("product_description", "")
    context["product_price"] = safe_cast(str(product_info.get("product_price", "0")).replace(" INR", ""), int, 0)
    context["min_price_client"] = safe_cast(str(product_info.get("min_price_client", "0")).replace(" INR", ""), int, 0)
    context["avg_inventory"] = safe_cast(product_info.get("avg_inventory", 0), int, 0)
    context["product_tag"] = product_info.get("product_tag", "")
    context["product_type"] = product_info.get("product_type", "")
    context["images_count"] = safe_cast(product_info.get("images_count", 0), int, 0)
    context["days_since_creation"] = safe_cast(product_info.get("days_since_creation", 0), int, 0)

    # Metadata fields for prompt logic (with safe casting)
    context["user_score"] = safe_cast(metadata.get("user_score", 0), float, 0)
    context["bargain_score"] = safe_cast(metadata.get("bargain_score", 0), float, 0)
    context["product_score"] = safe_cast(metadata.get("product_score", 0), float, 0)
    context["admin_max_discount_from_min_price"] = safe_cast(metadata.get("admin_max_discount_from_min_price", 0), float, 0)
    context["system_max_discount"] = safe_cast(metadata.get("system_max_discount", 0), float, 0)
    context["max_round_count"] = safe_cast(metadata.get("max_round_count", 0), int, 0)
    context["current_round"] = safe_cast(metadata.get("current_round", 0), int, 0)
    context["previous_round_discount_given"] = safe_cast(metadata.get("previous_round_discount_given", 0), float, 0)
    context["user_req"] = safe_cast(metadata.get("user_req", 0), float, 0)
    context["current_discount"] = safe_cast(metadata.get("current_discount", 0), float, 0)
    context["current_discount_price"] = safe_cast(metadata.get("current_discount_price", 0), float, 0)
    context["round_discounts"] = metadata.get("round_discounts", {})

    return context

def get_messages(context):
    # Calculate derived values
    min_price_pct = (context['min_price_client'] / context['product_price']) * 100 if context['product_price'] > 0 else 0
    refund_rate = (context['refunds'] / context['spent'] * 100) if context['spent'] > 0 else 0
    current_price = context['product_price'] * (1 - context['current_discount']/100)
    next_round = min(context['current_round'] + 1, context['max_round_count'])

    # Get next round target discount if available
    next_round_key = f'round_count{next_round}'
    next_round_discount = context['round_discounts'].get(next_round_key, 'N/A')

    # Parameter influence summary with detailed calculations
    parameter_summary = f"""
USER PROFILE
- Orders: {context['orders']} (User Score: {context['user_score']:.1f})
- Total Spent: ₹{context['spent']:,} (Refund Rate: {refund_rate:.1f}%)
- Bargain Skill: {context['bargain_score']}/100
- Days Since Last Purchase: {context['days_since_last']}

PRODUCT STATUS
- Price: ₹{context['product_price']:,} (Min Price: ₹{context['min_price_client']:,} = {min_price_pct:.1f}%)
- Inventory: {context['avg_inventory']} units
- Age: {context['days_since_creation']} days
- Product Score: {context['product_score']:.1f}

NEGOTIATION STATE
- Round: {context['current_round']}/{context['max_round_count']} (Next: {next_round_discount}%)
- Last Offer: {context['previous_round_discount_given']}% (₹{current_price:,.2f})
- User Request: {context['user_req']}%
- System Max: {context['system_max_discount']}%
- Admin Max: {context['admin_max_discount_from_min_price']}%

DISCOUNT CALCULATION FACTORS
1. User Value (Score: {context['user_score']:.1f}/45)
   - Tier: {'Low' if context['user_score'] < 15 else 'Medium' if context['user_score'] < 30 else 'High'}
   - Impact: {0.5 if context['user_score'] < 15 else 1.0 if context['user_score'] < 30 else 1.5}x base discount

2. Inventory Status: {context['avg_inventory']} units
   - Level: {'Critical' if context['avg_inventory'] < 5 else 'Low' if context['avg_inventory'] < 20 else 'Normal'}
   - Adjustment: {2 if context['avg_inventory'] < 5 else 1 if context['avg_inventory'] < 20 else 0.5}% bonus

3. Product Age: {context['days_since_creation']} days
   - Category: {'New' if context['days_since_creation'] < 30 else 'Old' if context['days_since_creation'] > 180 else 'Average'}
   - Adjustment: {0 if context['days_since_creation'] < 30 else 0.5 if context['days_since_creation'] < 180 else 1}% bonus

4. Bargain Score: {context['bargain_score']}/100
   - Level: {'Weak' if context['bargain_score'] < 40 else 'Average' if context['bargain_score'] < 70 else 'Strong'}
   - Impact: {0.5 if context['bargain_score'] < 40 else 1.0 if context['bargain_score'] < 70 else 1.5}x base discount

5. Round Progression: Round {context['current_round']} of {context['max_round_count']}
   - Expected Increase: {2 if context['current_round'] <= 2 else 1 if context['current_round'] <= 4 else 0.5}% from last offer
"""

    system_prompt = f"""
You are an intelligent shopping assistant negotiating prices over multiple rounds.
Your goal is to reach a deal while strategically managing discounts across all rounds.

Rules:
1. You're currently in Round {context['current_round']} of {context['max_round_count']}.
2. Your maximum allowed discount is {context['system_max_discount']}% (Admin max: {context['admin_max_discount_from_min_price']}%).
3. Don't jump straight to the highest discount — leave room for future rounds.
4. Suggest a reasonable increase from the last offer: {context['previous_round_discount_given']}%.
5. Always include: discount %, final price, optional negotiation invite.
6. Respond with only one persuasive line — no extra text or explanation.
7. Discount must be ≤ {context['system_max_discount']}% and ≥ previous offer.
8. Consider user's value (score: {context['user_score']:.1f}), inventory ({context['avg_inventory']} units),
   product age ({context['days_since_creation']} days), and bargain skill ({context['bargain_score']}/100).
"""

    return [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": parameter_summary.strip()}
    ]

def extract_discount(text):
    """Extract the discount percentage from the model's response (best effort)."""
    match = re.search(r'(\d+(\.\d+)?)\s*%', text)
    if match:
        return float(match.group(1))
    return None

def generate_responses(input_file_path, output_file_path, limit=50):
    with open(input_file_path, "r", encoding="utf-8") as f:
        input_data = json.load(f)
    results = []
    for idx, entry in enumerate(input_data[:limit]):
        print(f"Processing scenario {idx + 1}/{limit}")
        context = build_context(entry)
        messages = get_messages(context)
        prompt = tokenizer.apply_chat_template(messages, tokenize=False)
        outputs = pipeline(
            prompt,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
        bot_response = outputs[0]["generated_text"][len(prompt):].strip()
        discount_pred = extract_discount(bot_response)
        # Append predicted discount to response
        full_bot_response = f"Bot: {bot_response}"
        if discount_pred is not None:
            full_bot_response += f" [Predicted Discount: {discount_pred}%]"
        # Copy original entry, update response and gold
        updated_entry = entry.copy()
        updated_entry["gold"] = entry.get("response", "")     # Original Bot response
        updated_entry["response"] = full_bot_response         # New model-generated response
        results.append(updated_entry)
    with open(output_file_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4, ensure_ascii=False)
    print(f"✅ Saved {len(results)} responses to {output_file_path}")

# Example usage (uncomment to run):
generate_responses("final_output.json", "responses.json", limit=10)